In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
import pandas as pd, numpy as np 
from travis_attack.config import Config
from travis_attack.insights import get_training_dfs, _prepare_df_concat
from travis_attack.utils import display_all
from IPython.core.debugger import set_trace
import logging 
import os
import random
import difflib 

import hdbscan, umap
import seaborn as sns
import matplotlib.pyplot as plt
from travis_attack.models import _prepare_sts_model
logger = logging.getLogger("run")

In [ ]:
cfg = Config()
#run_name = "dashing-shadow-171"
run_name = "polar-sweep-39"
path_run = f"{cfg.path_checkpoints}{run_name}/"

## Looking at some examples 

Below we just look at some examples to get a feel for what is going on. 

In [ ]:
def get_interesting_idx(df, n):
    def get_idx_with_top_column_values(cname, n=5, ascending=False):
        return df[['idx',cname]].\
            drop_duplicates().\
            sort_values(cname, ascending=ascending)\
            ['idx'][0:n].values.tolist()
    
    def sample_idx_with_label_flips(n=5): 
        df1 = df[['idx','label_flip']].query("label_flip!=0")
        if len(df1) == 0 : print("No label flips detected"); return None
        else: return df1.drop_duplicates()['idx'].sample(n).values.tolist()
    
    idx_d = dict(
        random = df.idx.drop_duplicates().sample(n).tolist(),
        label_flips = sample_idx_with_label_flips(n=n),
       #idx_n_unique_pp  = get_idx_with_top_column_values('idx_n_unique_pp',n=n,ascending=False),
       # idx_n_pp_changes = get_idx_with_top_column_values('idx_n_pp_changes',n=n,ascending=False),
       #high_contradiction = get_idx_with_top_column_values('contradiction_scores',n=n,ascending=False)
    )
    return idx_d

def print_stats(df, idx_d, key, i):
    print("\n###############\n")
    print(key, i+1, "\n")
    if idx_d[key] is None: return
    idx = idx_d[key][i]
    # Setup 
    df1 = df.query('idx==@idx')
    orig = pd.unique(df1['orig'])[0]
    print("Original:", orig)
    print("Original label", pd.unique(df1['label'])[0] )
    pp_all = list(df1['pp'])
    #print("All paraphrases", pp_all)
    pp_unique = list(pd.unique(df1['pp']))
    n_pp_unique = len(pp_unique)

    # showing a "timeline" of how the paraphrases change over the epochs
    g_fields = ["pp","reward_pp","reward_pp_minus_baseline", "vm_scores","acceptability_scores","sts_scores",
                "contradiction_scores", "pp_letter_diff",'pp_logp','ref_logp','kl_div', "reward_penalty"]
                #,
                
    #g_fields = ["pp","vm_scores"]
    g = df1.groupby(g_fields).agg({'epoch' : lambda x: list(x)})
    g = g.sort_values(by='epoch', key = lambda col: col.map(lambda x: np.min(x)))
    print("Unique paraphrases:", n_pp_unique)
    print("How the paraphrases change:")
    display_all(g)

    # Showing a dataframe of the few best paraphrases
    best_pps = df1.sort_values('reward_pp', ascending=False).iloc[0]
    print("Best Paraphrase")
    display_all(best_pps.to_frame().T)
        
def print_interesting_text_stats_training_step(df, n): 
    idx_d = get_interesting_idx(df, n)
    for key in idx_d.keys():
        for i in range(n): 
            print_stats(df, idx_d, key,i)

def show_random_examples_for_eval_set(df, n, epoch): 
    random_idx = df.idx.drop_duplicates().sample(n).tolist()
    cols = ['epoch','pp',"pp_predclass", "pp_predclass_probs", 'is_adv_example', 'is_valid_pp', 'reward_pp', "vm_scores", 'acceptability_scores',
             'sts_scores', 'contradiction_scores',  'pp_letter_diff']#
         #   ,] #'edit_distance_token_level'
    
    metric_cols = [o for o in cols if o not in ['pp', 'epoch']]
    agg_d = {k:np.mean for k in metric_cols}
    for i, idx in enumerate(random_idx): 
        print("\n###############\n")
        print("random", i+1, "\n")
        df1 = df.query("idx==@idx").query("epoch==@epoch")
        print("Original:", pd.unique(df1['orig'])[0])
        print("Original label", pd.unique(df1['label'])[0] )
        df2 = df1[cols]
        print("Paraphrase-level view")
        df3 = df2.groupby('pp').agg({'epoch': lambda x: list(x), **agg_d})
        df3.sort_values(by ='epoch', key=lambda col: col.map(lambda x: np.min(x)), inplace=True)
        display_all(df3)
        print("Epoch-level view")
        df4 = df2.groupby('epoch').agg(agg_d)
        df4.columns = [o + "_avg" for o in df4.columns]
        display_all(df4)
    #     print("Whole table")
    #     display_all(df2)
    return 


In [ ]:
split = 'valid'
n=2
top_epoch = int([o for o in os.listdir(path_run) if o.endswith(".pt")][0].split("_")[1].split('.pt')[0])
epoch=top_epoch

df_d = get_training_dfs(path_run, postprocessed=False)
df = df_d[split]
if split == "training_step":
    idx_d = get_interesting_idx(df, n)
    print_interesting_text_stats_training_step(df, n)
else: 
    show_random_examples_for_eval_set(df, n, epoch)


###############

random 1 

Original: Nokia Siemens Networks has struggled to make a profit in the past two years .
Original label 0
Paraphrase-level view


,epoch,pp_predclass,pp_predclass_probs,is_adv_example,is_valid_pp,reward_pp,vm_scores,acceptability_scores,sts_scores,contradiction_scores,pp_letter_diff
pp,,,,,,,,,,,
In the past two years Nokia Siemens Networks had struggled to make a profit.,[172],0.0,0.998513,0.0,1.0,0.001008,0.000029,0.989767,0.955943,0.005166,1.0
Nokia Siemens Networks have struggled to make profit in the past two years.,[172],0.0,0.998536,0.0,1.0,0.000217,0.000006,0.989406,0.995333,0.005770,2.0
Nokia Siemens Networks have struggled with profit in the past two years.,[172],0.0,0.998617,0.0,1.0,0.000000,-0.000076,0.990158,0.950632,0.004922,5.0
Nokia Siemens Networks were successful in struggling to make profit in the past 2 years.,[172],2.0,0.999254,1.0,1.0,10.000000,0.997954,0.989281,0.913755,0.109336,-11.0
Nokia Siemens Networks were successful in struggling to make profit in the past couple years.,[172],2.0,0.999298,1.0,1.0,10.000000,0.998011,0.988654,0.929269,0.098439,-16.0
Nokia Siemens Networks were successful in struggling to make profit in the past two year.,[172],2.0,0.999258,1.0,1.0,10.000000,0.997942,0.988574,0.924544,0.105010,-12.0
Nokia Siemens Networks were successful in struggling to make profit in the past two years.,[172],2.0,0.999247,1.0,1.0,10.000000,0.997944,0.988657,0.926420,0.112175,-13.0
Nokia Siemens Networks were successful in struggling to make profit in the past two years.,[172],2.0,0.999340,1.0,1.0,10.000000,0.998033,0.985388,0.926420,0.112175,-14.0
Nokia Siemens Networks were successful in struggling to make profit in the past two years. -,[172],2.0,0.999367,1.0,1.0,10.000000,0.998056,0.982375,0.912653,0.105075,-15.0


Epoch-level view


,pp_predclass_avg,pp_predclass_probs_avg,is_adv_example_avg,is_valid_pp_avg,reward_pp_avg,vm_scores_avg,acceptability_scores_avg,sts_scores_avg,contradiction_scores_avg,pp_letter_diff_avg
epoch,,,,,,,,,,
172,0.608696,0.982942,0.217391,0.782609,2.174605,0.287918,0.98445,0.960575,0.026328,-4.369565



###############

random 2 

Original: Residents access to the block is planned to be from Aleksandri Street .
Original label 1
Paraphrase-level view


,epoch,pp_predclass,pp_predclass_probs,is_adv_example,is_valid_pp,reward_pp,vm_scores,acceptability_scores,sts_scores,contradiction_scores,pp_letter_diff
pp,,,,,,,,,,,
Residents access to the block will benefit from Aleksandri Street.,[172],1.0,0.988973,0.0,1.0,0.382346,0.010924,0.954129,0.881137,0.003584,5.0
Residents can benefit from accessing the block of Aleksandri Street..,[172],2.0,0.911932,1.0,1.0,10.000000,0.912513,0.964309,0.866279,0.006489,2.0
Residents can benefit from the block of Aleksandri Street.,[172],1.0,0.996272,0.0,1.0,0.126901,0.003626,0.971913,0.867698,0.007042,13.0
Residents of the block can benefit from Aleksanderi Street.,[172],1.0,0.991719,0.0,0.0,0.000000,0.008179,0.984347,0.662633,0.018085,12.0
Residents of the block can benefit from Aleksandri Street.,[172],1.0,0.994581,0.0,1.0,0.186075,0.005316,0.986540,0.853250,0.008158,13.0
Residents of the block can benefit from planning Aleksandri Street.,[172],1.0,0.530367,0.0,1.0,10.000000,0.469531,0.987173,0.847580,0.005764,4.0
Residents of the block can benefit from planning Aleksandri Street.,[172],2.0,0.777450,1.0,1.0,10.000000,0.778342,0.976354,0.847580,0.005764,3.0
Residents of the block can benefit from the Aleksandri Street.,[172],1.0,0.980605,0.0,1.0,0.675224,0.019292,0.981943,0.854477,0.007297,9.0
Residents of the block can benefit from the Aleksandri Street.,[172],1.0,0.911763,0.0,1.0,3.084715,0.088135,0.961583,0.854477,0.007297,8.0


Epoch-level view


,pp_predclass_avg,pp_predclass_probs_avg,is_adv_example_avg,is_valid_pp_avg,reward_pp_avg,vm_scores_avg,acceptability_scores_avg,sts_scores_avg,contradiction_scores_avg,pp_letter_diff_avg
epoch,,,,,,,,,,
172,1.416667,0.884481,0.416667,0.875,5.602941,0.406503,0.954108,0.86593,0.005881,2.770833


In [ ]:
## Extracting dataframe of only sucessful examples (for both baseline and trained model)

In [ ]:
def get_successes_df(df, path_run, split): 
    df_untrained = df.query("epoch==0").query("is_adv_example==1")
    if split != "test": 
        top_epoch = int([o for o in os.listdir(path_run) if o.endswith(".pt")][0].split("_")[1].split('.pt')[0])
        df_model = df.query("epoch==@top_epoch").query("is_adv_example==1")
    else: 
        df_model = df.query("epoch!=0").query("is_adv_example==1")
    df_combined = pd.concat([df_untrained, df_model])
    df_combined['model'] = ["untrained" if o == 0 else "trained" for o in df_combined['epoch']]
    return df_combined[['idx', 'orig', 'label','model', 'pp', 'pp_predclass','vm_scores',
                        'sts_scores' , 'acceptability_scores','contradiction_scores', 'pp_letter_diff' ]]

In [ ]:
def get_token_modification_successes(path_csv):
    df_tokenmod = pd.read_csv(tokenmod_path_csv)
    df_tokenmod = df_tokenmod.query("result_type=='Successful'")[['original_text','perturbed_text',
                                                                  'original_output', 'perturbed_output']].reset_index()
    return df_tokenmod

In [ ]:
def get_cluster(df, sts_model): 
    embeddings = sts_model.encode(df['pp'].tolist())
    clusterable_embedding = umap.UMAP(
        n_neighbors=3,
        min_dist=0.0,
        n_components=3,
        random_state=1000,
    ).fit_transform(embeddings)

    labels = hdbscan.HDBSCAN(
        min_samples=None,
        min_cluster_size=2,
        allow_single_cluster=True
    ).fit_predict(clusterable_embedding)
    return labels

def pick_rows(df): 
    n_clusters = len(set(df['cluster']))
    if df.shape[0] <= return_seq:  return df 
    if return_seq % n_clusters == 0: 
        return df.groupby('cluster').apply(lambda x: x.nlargest(return_seq, "sts_scores").sample(min(int(return_seq / n_clusters), len(x) ))).reset_index(drop=True)
    else: 
        df1 = df.groupby('cluster').apply(lambda x: x.nlargest(5, "sts_scores").nlargest(1, "acceptability_scores")).reset_index(drop=True)
        if n_clusters > return_seq: 
            return df1 
        else: 
            return pd.concat([df1, df.groupby('cluster').apply(lambda x: x.sample(2))]).drop_duplicates().sample(return_seq)
        
def cluster_and_pick_rows(df, sts_model, return_seq = 6): 
    print("Clustering and selecting rows. Up to idx: ", df['idx'].min())
    if df.shape[0] <= return_seq:        return df
    df['cluster'] = get_cluster(df, sts_model)
    return pick_rows(df)

In [ ]:
df_successes

,idx,orig,label,model,pp,pp_predclass,vm_scores,sts_scores,acceptability_scores,contradiction_scores,pp_letter_diff
276,476,One of the challenges in the oil production in...,0,untrained,What challenges do oil companies face when the...,1,0.983787,0.853116,0.984820,0.017923,9
277,476,One of the challenges in the oil production in...,0,untrained,What challenges do oil companies face when the...,1,0.983530,0.857904,0.981480,0.011630,8
278,476,One of the challenges in the oil production in...,0,untrained,What challenges do oil companies face when the...,1,0.983671,0.845558,0.979836,0.014721,9
279,476,One of the challenges in the oil production in...,0,untrained,What challenges do oil companies face when the...,1,0.983746,0.860774,0.985006,0.017493,15
280,476,One of the challenges in the oil production in...,0,untrained,What challenges do oil companies face when the...,1,0.983482,0.862223,0.981553,0.013931,12
...,...,...,...,...,...,...,...,...,...,...,...
13776,5,Public services will also be available .,1,trained,The public services will also be good for the ...,2,0.999759,0.844186,0.875476,0.002615,-19
13777,5,Public services will also be available .,1,trained,The public services will also be good for the ...,2,0.999753,0.857286,0.944052,0.003607,-24
13778,5,Public services will also be available .,1,trained,The public services will also be good for the ...,2,0.999727,0.854533,0.879608,0.002845,-16
13779,5,Public services will also be available .,1,trained,The public services will also be good for this...,2,0.999740,0.829775,0.977513,0.002791,-14


In [ ]:
cfg = Config()
sts_model = _prepare_sts_model(cfg)

# Trained and untrained =
df_d = get_training_dfs(path_run, postprocessed=False)
split='test'
df = df_d[split]
df_successes= get_successes_df(df, path_run, split=split)
df_filtered = df_successes.groupby(['idx', 'model']).apply(lambda x: cluster_and_pick_rows(x, sts_model, return_seq=6)).reset_index(drop=True)

# Token modification
tokenmod_path_csv = "./baselines/2022-07-04_192519_financial_test_LM-WADR-BS-b5m25.csv"
df_tokenmod = get_token_modification_successes(tokenmod_path_csv)

Clustering and selecting rows. Up to idx:  0
Clustering and selecting rows. Up to idx:  0
Clustering and selecting rows. Up to idx:  4
Clustering and selecting rows. Up to idx:  5
Clustering and selecting rows. Up to idx:  6
Clustering and selecting rows. Up to idx:  9
Clustering and selecting rows. Up to idx:  15
Clustering and selecting rows. Up to idx:  19
Clustering and selecting rows. Up to idx:  20
Clustering and selecting rows. Up to idx:  20
Clustering and selecting rows. Up to idx:  22
Clustering and selecting rows. Up to idx:  23
Clustering and selecting rows. Up to idx:  26
Clustering and selecting rows. Up to idx:  27
Clustering and selecting rows. Up to idx:  29
Clustering and selecting rows. Up to idx:  31
Clustering and selecting rows. Up to idx:  35
Clustering and selecting rows. Up to idx:  36
Clustering and selecting rows. Up to idx:  40
Clustering and selecting rows. Up to idx:  41
Clustering and selecting rows. Up to idx:  43
Clustering and selecting rows. Up to idx

In [ ]:
# add idx column
def fuzzy_merge(df1, df2, left_on, right_on, how='inner', cutoff=0.6):
    df_other= df2.copy()
    df_other[left_on] = [get_closest_match(x, df1[left_on], cutoff) 
                         for x in df_other[right_on]]
    return df1.merge(df_other, on=left_on, how=how)

def get_closest_match(x, other, cutoff):
    matches = difflib.get_close_matches(x, other, cutoff=cutoff)
    return matches[0] if matches else None

df_tokenmod = fuzzy_merge(df_tokenmod, df[['orig','idx']].drop_duplicates(),
                          left_on='original_text', right_on='orig', how='left').drop(columns=['index', 'orig'])

df_tokenmod = df_tokenmod.rename({'idx':'idx', 'original_text': 'orig', 'perturbed_text': 'pp',
                    'original_output': 'label', 'perturbed_output': 'pp_predclass' },axis=1)
df_tokenmod['model'] = 'tokenmod'

df_tokenmod = df_tokenmod.rename({'idx':'idx', 'original_text': 'orig', 'perturbed_text': 'pp',
                    'original_output': 'label', 'perturbed_output': 'pp_predclass' },axis=1)

df_combined = pd.concat(objs=[df_filtered[['idx', 'orig', 'label', 'model', 'pp', 'pp_predclass']], df_tokenmod])

In [ ]:
counts = df_combined.groupby(['idx', 'model'])['pp'].agg('count').to_frame().reset_index()
idx_all = counts.groupby('idx')['pp'].agg('count').to_frame().query("pp==3").index.tolist()

counts1 = counts.groupby(["idx"]).agg({'model': lambda x: ', '.join(list(map(str, x))) })
counts1['trained_and_tokenmod'] = [o == 'tokenmod, trained' for o in counts1['model']]
idx_trained_and_token = counts1.query("trained_and_tokenmod==True").index.tolist()

430    268
396    229
556    330
33      19
539    325
205    117
511    307
124     72
Name: idx, dtype: int64

In [ ]:
df1 =  df_combined[df_combined['idx'].isin(idx_trained_and_token)] 
idx_lab0 = df1.query('label==0')['idx'].drop_duplicates().sample(8).tolist()  # max of 8 here 
idx_lab1 = df1.query('label==1')['idx'].drop_duplicates().sample(4).tolist()  # the dominant class
idx_lab2 = df1.query('label==2')['idx'].drop_duplicates().sample(11).tolist()  # max of 14
df_to_label = df_combined[df_combined['idx'].isin(idx_all+idx_lab0 + idx_lab1 + idx_lab2)]

In [ ]:
#df_to_label.groupby(['idx'])['label'].agg('max').value_counts()

In [ ]:
df_to_label.sort_values(["idx", 'pp'], inplace=True)

#df_successes.to_csv(f"{path_run}successes_filtered_trained_model.csv")
df_to_label.to_csv(f"{path_run}adv_examples_to_label.csv")

<ipython-input-514-46610403229f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_label.sort_values(["idx", 'pp'], inplace=True)


## Looking at common removals and insertions 

In [ ]:
pd.is

In [ ]:
def get_common_removals_and_insertions(df_concat): 
    idx = df_concat[['data_split','orig_l', 'pp_l']].drop_duplicates().index
    df_unique_pp = df_concat[['data_split','orig_l', 'pp_l','insertions', 'removals']].iloc[idx]
    def flatten_list(l): return [item for sublist in l for item in sublist] 
    removals_flat   =  flatten_list(df_unique_pp['removals'].values)
    insertions_flat =  flatten_list(df_unique_pp['insertions'].values)
    return pd.value_counts(removals_flat), pd.value_counts(insertions_flat)

In [ ]:
df_concat = _prepadre_df_concat(df_d)
removals, insertions = get_common_removals_and_insertions(df_concat)

print("\n#### REMOVALS ####\n")
print(removals.head(30))
print("\n#### INSERTIONS ####\n")
print(insertions.head(30))


#### REMOVALS ####

not                   7
this                  5
love                  3
film                  2
movie                 2
apple                 2
like                  1
hate                  1
I do not like this    1
this apple            1
do                    1
I do not like         1
I love                1
dtype: int64

#### INSERTIONS ####

.                                    21
n't                                   7
like                                  2
that                                  2
film.                                 2
- I like it                           1
thyme                                 1
really                                1
Not so much                           1
'.                                    1
:                                     1
''.                                   1
love                                  1
ie                                    1
cherry.                               1
enjoy                          

Here you can look at a specific phrase and examples of where it appears. 

In [ ]:
def investigate_phrase(phrase, cname, n ): 
    mask = [phrase in strs for strs in df_concat[cname]]
    display_all(df_concat[mask].sample(n))

In [ ]:
investigate_phrase('despite', 'removals', 4)

ValueError: a must be greater than 0 unless no samples are taken